In [3]:
import os
import argparse
!pip install mediapipe
import cv2
import mediapipe as mp

In [5]:
def process_img(img, face_detection):
    H, W, _ = img.shape

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out = face_detection.process(img_rgb)

    if out.detections is not None:
        for detection in out.detections:
            location_data = detection.location_data
            bbox = location_data.relative_bounding_box

            x1, y1, w, h = bbox.xmin, bbox.ymin, bbox.width, bbox.height

            x1 = int(x1 * W)
            y1 = int(y1 * H)
            w = int(w * W)
            h = int(h * H)

            img[y1:y1 + h, x1:x1 + w, :] = cv2.blur(img[y1:y1 + h, x1:x1 + w, :], (30, 30))

    return img


In [7]:
output_dir = './output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [8]:
# Change this path to your image file
image_path = '/kaggle/input/image1/data1.jpg'

mp_face_detection = mp.solutions.face_detection

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    img = cv2.imread(image_path)
    if img is not None:
        img = process_img(img, face_detection)
        cv2.imwrite(os.path.join(output_dir, 'output_image.png'), img)
        print("Saved blurred image.")
    else:
        print("Error: Could not read the image.")


Saved blurred image.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751342221.291056     108 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [11]:
# Change this path to your video file
video_path = '/kaggle/input/video1/Video1.mp4'

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

if not ret:
    print("Error: Cannot read video.")
else:
    output_video = cv2.VideoWriter(os.path.join(output_dir, 'output_video.mp4'),
                                   cv2.VideoWriter_fourcc(*'MP4V'),
                                   25,
                                   (frame.shape[1], frame.shape[0]))

    with mp.solutions.face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
        while ret:
            frame = process_img(frame, face_detection)
            output_video.write(frame)
            ret, frame = cap.read()

    cap.release()
    output_video.release()
    print("Saved blurred video.")


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
W0000 00:00:1751342634.316758     133 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/box_filter.dispatch.cpp:446: error: (-215:Assertion failed) !_src.empty() in function 'boxFilter'
